In [2]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-08-18 12:58:48--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py.1’

minsearch.py.1      100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-08-18 12:58:48 (31.5 MB/s) - ‘minsearch.py.1’ saved [3832/3832]



In [3]:
import minsearch
import json

In [4]:
with open('documents.json', 'rt') as f_in:     # rt = read text, f_in = file input
    docs_raw = json.load(f_in)

In [5]:
documants = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documants.append(doc)

In [5]:
documants[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe next cohort starts in Jan 2025. More info at DTC Article.\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start？',
 'course': 'data-engineering-zoomcamp'}

In [6]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [7]:
index.fit(documants)

In [15]:
import os
from openai import OpenAI

client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])


In [53]:
def search(query):
    boost =  {"question": 3.0,  "section": 0.5}

    results = index.search(
        query= query,
        boost_dict= boost,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        num_results= 5
        )
    return results

In [54]:
def build_prompt(query, search_results):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.

    QUESTION: {question}

    CONTEXT: 
    {context}
    """.strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [55]:
def llm(promot):
    response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": promot}]
    )

    return response.choices[0].message.content

In [58]:
def rag(query):
    search_results = search(query)
    promot = build_prompt(query, search_results)
    answer = llm(promot)
    return answer

In [60]:
query = 'how do i run kafka?'
print(rag(query))

To run Kafka, the specific instructions depend on your system setup and the programming language you are using. Here are some relevant steps based on the CONTEXT provided:

1. **Using Java:**
   - In the project directory, you can run a Java Kafka producer or consumer with the following command:
     ```shell
     java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java
     ```
     Replace `<jar_name>` with the actual name of the jar file.

2. **Using Python:**
   - If you're running a Python Kafka producer or consumer, you should first create and activate a virtual environment, then install the necessary packages:
     1. Create a virtual environment (you only need to do this once):
        ```shell
        python -m venv env
        ```
     2. Activate the virtual environment:
        - On macOS or Linux:
          ```shell
          source env/bin/activate
          ```
        - On Windows:
          ```shell
          env\Scripts\activate
